In [ ]:
# The preliminary notebook for testing/developing wrsamp. Will delete before merging. 

In [ ]:
# This cell will go into _wrsamp.py


    
    

In [48]:
import numpy as np
a=np.zeros([10,3])
m, n = np.shape(a)


In [19]:
WFDBfields = ['height', 'weight', 'age']

final = dict.fromkeys(WFDBfields, [])
final

{'age': [], 'height': [], 'weight': []}

In [18]:
help(dict.fromkeys)

Help on built-in function fromkeys:

fromkeys(iterable, value=None, /) method of builtins.type instance
    Returns a new dict with keys from iterable and values equal to value.



In [20]:
for i in final:
    print(i)

age
weight
height


In [49]:
if np.shape(a) == (10, 3):
    print('he')

he
